# Tackling The Generative learning Trilemma with Denoising Diffusion GANs

## Contributions

+ 생성모델의 큰 문제인 퀄리티, 다양성, 속도 세가지 단점을 diffusion model과 GAN을 결합하여 해결하였다.


## Introduction

<img src="https://drive.google.com/uc?id=1PhLm3Quz8ujgGS8Wycjjm0RyYO6z4LbP" height=300>

+ 생성모델의 3가지  문제
    + high quality sampling
    + mode coverage & sample diversity
    + sampling speed & computational cost

GAN, VAE, normalizaint flow, diffusion 등의 여러 모델이 나왔지만, 모든 부분에서 우수한 모델은 없었다.

denoising diffusion 모델에 tackle을 걸 것이다!

__diffusion process의 reverse 과정을 Gaussian distribution으로 가정하기 때문에 극히 작은 step의 denoising을 할 수 있다__

image synthesis 분야에서 multimodal distribution이 자연스럽다.

## Denoising Diffusion GANs

forward process(가정)
+ $q(x_t|x_{t-1}) = \mathcal{N}(x_t;\sqrt{1-\beta_t}x_{t-1}, \beta_t\mathbf{I})$  


reverse process(근사)  
+ $p_{\theta}(x_{t-1}|x_t) = \mathcal{N}(x_{t-1};\mu_\theta(x_t, t), \sigma^2_t\mathbf{I})$

### Multimodal denoising distributions for large denoising steps

+ multimodal distribution : 여러개의 최빈값을 갖는 분포

<img src="https://drive.google.com/uc?id=1hYak1uhu0tBLDjP1NKC9qehncEMSY071" height=250>

diffusion model은 결국 $q(x_{t-1}|x_t)$를 Gaussian distribution으로 가정하고, 이를 파라미터화 하여 근사해나가는 과정이다.

<img src="https://drive.google.com/uc?id=16o31ZKwJZXt_rMvMAQqD5GhfFg00oMSP" height=300>

#### why single denoising process is Gaussian?
1. 만약 variance $\beta_t$가 아주 작다면, 베이즈정리 에서 $q(x_t|x_{t-1})$값이 지배적이게 된다고 한다. (why?) 또한 forward는 Gaussian을 가정하였으므로 $q(x_{t-1}|x_t)$역시 Gaussian이라고 한다.

2. denoising 과정에서 $q(x_t)$ 분포 자체가 Gaussian이라면 $q(x_{t-1}|x_t)$가 Gaussian 이 성립한다. 이러한 방법은 VAE encoder를 이용하는 방법 등이 있지만, 그리 완벽한 성능을 보이고있지 않다.

기존의 diffusion model들은 이러한 가정하에 approximation을 진행하고 있고, 그것이 옳다.  
 __다만, 작은 $\beta$를가진 수천번의 step이 필요해진다.__

__해당 논문에선 denosing step을 키우고(보폭을 키운다는 뜻), denoising data distribution을 non-Gaussian으로 가정한다.(Fig.2)__

### Modeling distribution with conditional GANs

기존과 다른점은, $T \leq 8$으로 매우 작은 step이다.    
conditional GAN generator를 $p_\theta(x_{t-1}|x_t)$와 같이 정의하였을 때
목적함수는 다음과 같다.  
$$\min_\theta \sum_{t \geq 1}\mathbb{E}_{q(x_t)}[D_{adv}(q(x_{t-1}|x_t)||p_\theta(x_{t-1}|x_t))]$$

$D_{adv}$로 여러가지 거리공간을 사용한다. 여기선 f-divergence를 사용하였다.(backward KL divergence)

$D_\phi$ : Discriminator  
$G_\theta$ : Generator

#### Training Discriminator
$D_\phi(x_{t-1}, x_t, t) : \mathbb{R}^N \times \mathbb{R}^N \times \mathbb{R} \rightarrow [0, 1]$

+ objective function
$$\min_\phi \sum_{t \leq1}\mathbb{E}_{q(x_t)} \big[\mathbb{E}_{q(x_{t-1}|x_t)}[-\log(D_\phi(x_{t-1}, x_t, t))] + \mathbb{E}_{p_\theta(x_{t-1}|x_t)}[-\log(1 - D_\phi(x_{t-1}, x_t, t))] \big]$$

<img src="https://drive.google.com/uc?id=167i18h7wBn9eJt79eAVnfwk1JxrtT68j" height=500>

즉, $x_0$로 $x_{t-1}$을 뽑고 그것으로 $x_t$를 샘플링해주면 된다.  
(이렇게 하는 이유는 true reverse process를 아직 정의해놓지 않았기 때문이다.)

#### Parametrizing the implicit denoising model  
GAN 장점이자 단점이 바로 분포를 모르는 상태로 모델링한다는 것이다.

+ 기존의 denoising 방식(DDPM)  
$p_\theta(x_{t-1}|x_t) := q(x_{t-1}|x_t, x_0 = f_\theta(x_t, t))$  
1. $x_t$를 모델에 넣어 해당 time step의 noise인 $\epsilon_\theta(x_t)$을 얻어낸다.
2. 여기에 사전에 정의한 $\alpha$와 $x_t, \epsilon_\theta(x_t)$를 이용해  $x_0$를 얻어낸다.
3. $x_t, x_0$ 두개를 통해 Gaussian distribution의 mean, variance 얻어내고 여기에서 $x_{t-1}$을 샘플링 해낸다.

+ 본 논문 denoising 방식(DDGAN)  
$p_\theta(x_{t-1}|x_t) := \int p_\theta(x_0|x_t)q(x_{t-1}|x_t, x_0)dx_0 = \int p(z)q(x_{t-1}|x_t, x_0 = G_\theta(x_t, z, t))dz$  
1. random noise $z$를 얻어낸다. $\mathcal{N}(z;0, \mathbf{I})$
2. $x_t$와 $z$를 GAN에 넣어 $x_0$을 얻어낸다.
3. $x_t, x_0$ 두개를 통해 Gaussian distribution의 mean, variance 얻어내고 여기에서 $x_{t-1}$을 샘플링 해낸다.

<img src="https://drive.google.com/uc?id=1VhP_7Z5Xtw4hqgqIqv3GvvA6p1GZzmJy" height=300>

#### Advantage over DDPM
1. $p_\theta$를 DDPM과 비슷한 형태로 parameter화 하였다.
    + 비슷한 inductive bias를 가져갈 수 있다.
2. 각 time step에 $x_0$만 예측할 수 있으면 된다. (문제 정의가 더 쉬워진다.)
    + 기존에는 t-1 -> t에 해당하는 1step noise를 예측해내야 했으나, 해당 모델은 t step 전체를 예측하게 됨.

__$x_0$를 얻어낼 때 GAN을 사용하는 것이 DDPM(U-Net)과의 가장 큰 차이점이고, 이를 통해 $p_\theta(x_{t-1}|x_t)$가 multimodal distribution이 되는 열쇠이다.__

$x_t \rightarrow x_0$에서
+ DDPM : deterministic (by only $\alpha$)
    + unimodal (Gaussian)
+ DDGAN : random (by $z$)
    + multimodal (GAN이기 때문에 분포는 모르지만, true denoising distribution을 따라서 sampling되게 된다. (Implicit))

#### Advantage over one-shot generator(GAN)

기존 GAN의 어려움
+ training instability,  mode collapse
+ 특정 clean sample들만 볼 때 Discriminator가 overfitting되기도한다.
+ 복잡한 분포로부터 바로 sampling 하는 것의 어려움.

장점  
1. 각 step마다 상대적으로 간단한 모델로 예측할 수 있게된다.  
$\because x_t$ is strong condition
2. Discriminator가 overfit될 가능성을 낮췄다.  
$\because$ diffusion process가 data의 분포를 smooth하게 해준다. -> less clean

## Experiments

T=4일 때 가장 좋았고 이것을 기준으로 다른 모델들과 비교하였다.

### Overcoming the Generative learning Trilemma

<img src="https://drive.google.com/uc?id=1kD1hNqKMgcGfZiytDe7YCaHwFPsBER7J" height=300>

trade-off를 상당히 해결하였음을 볼 수 있다.

<img src="https://drive.google.com/uc?id=1lf8dkxumm3feavbjqf6rIW_hEwMhV-K_" width=500>

sample diversity에 해당하는 recall score가 style gan보다 좋다.
(실제 데이터셋을 더욱 많이 cover한다.)

### Ablation studies

<img src="https://drive.google.com/uc?id=1-wJnuCbqnuspMj2XKWXRkHfOPxA_2SY_" height=250>

+ Number of denoising steps
    + T = 1 -> unconditional GAN
    + T = 4 : best model
+ Diffusion as data augmentation
    + forward process를 살펴보면, 원래 데이터에 noise가 들어간 것이 augmentation이라고 볼 수 있다. 이러한 데이터를 그냥 GAN에 적용하여 학습시켜봤다.
    + 결과 안좋음. -> DDGAN과 GAN은 다르다.
+ Parametrization for $p_\theta(x_{t-1}|x_t)$
    + direct denoising ($x_t \rightarrow x_{t-1}$
    + noise generation
+ Importance of latent variable
    + z를 제거해보았다. -> 안좋음
    + multimodal denoising distribution의 중요성!

### Additional studies

<img src="https://drive.google.com/uc?id=1JTIFWmaQQxYTcvn8bjvDuZ1w7H9JH22J" height=200>

우수한 Mode coverage를 가지고 있다.

<img src="https://drive.google.com/uc?id=1rOPbUwkkOBv4bkzl3dn1StlPVp6oXaKb" height=150>

sample quality역시 우수하다.